In [1]:
# Edge detection in video

import numpy as np
import cv2

video = cv2.VideoCapture('road_car_view.mp4')

while True:
    ret, or_frame = video.read()
    if not ret :
        video = cv2.VideoCapture('road_car_view.mp4')
        continue
        
    frame = cv2.GaussianBlur(or_frame, (5,5), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_y = np.array([18,94,140])
    upper_y = np.array([48, 255, 255])
    
    mask = cv2.inRange(hsv, lower_y, upper_y)
    edges = cv2.Canny(mask, 74, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50)
    
    if lines is not None:
        for line in lines:
            x1, y1, x2,y2 = line[0]
            cv2.line(frame, (x1,y1),(x2,y2),(0,255,0), 5)
    cv2.imshow("Frame", frame)
    cv2.imshow("edges", edges)
    if cv2.waitKey(1)==3:
        break
        
video.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [1]:
# Project Highway Vehicle Counter


import cv2
import math

class EuclideanDistTracker:
    def __init__(self):
        self.center_points = {} # store the center position of the object
        # keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0
        
    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []
        
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x+x+w)//2
            cy = (y+y+h)//2
            
            # Find out if the object was detected already
            
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])
                
                if dist <25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x,y, w,h, id])
                    same_object_detected = True
                    break
                    
            # New Object id detected we assign the id to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x,y, w,h, self.id_count])
                self.id_count +=1
                
        # Clean the dictionary by center point to remove IDs not used anymore
        
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _,_,_,_, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center
            
        # Update dictionary with IDs not used - remove
        self.center_points = new_center_points.copy()
        return objects_bbs_ids
                

In [2]:
import cv2
tracker = EuclideanDistTracker()
cap = cv2.VideoCapture('highway.mp4')

object_detector = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape
    
    roi = frame[340:720, 500:800]
    
    # part1 - Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254,255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100:
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x, y, w, h])
    
    # Part2 - Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN,2, (255,0,0),2)
        cv2.rectangle(roi, (x,y), (x+w, y+h),(0,255,0), 3)
    cv2.imshow("roi", roi)
    cv2.imshow("frame", frame)
    cv2.imshow("mask", mask)
    
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindow()

{1: (162, 60)}
{1: (162, 78)}
{1: (162, 79)}
{1: (163, 79)}
{2: (162, 122)}
{2: (161, 115)}
{2: (162, 136)}
{2: (163, 149)}
{2: (162, 144)}
{2: (160, 152)}
{2: (163, 174)}
{2: (160, 167), 3: (158, 148)}
{2: (157, 149), 3: (158, 148)}
{4: (159, 190)}
{4: (158, 199)}
{4: (161, 211)}
{4: (160, 222)}
{4: (156, 232)}
{4: (156, 245)}
{4: (156, 256)}
{4: (155, 269)}
{4: (154, 283)}
{4: (153, 299)}
{4: (151, 314)}
{4: (151, 330)}
{4: (150, 342)}
{4: (148, 351)}
{4: (145, 360)}
{4: (143, 369)}
{6: (207, 20)}
{6: (207, 24)}
{6: (207, 19)}
{6: (207, 30)}
{6: (207, 25)}
{6: (209, 26)}
{6: (208, 31)}
{6: (209, 34)}
{6: (209, 38)}
{6: (209, 41)}
{6: (208, 53)}
{6: (211, 35)}
{6: (209, 48)}
{6: (209, 51)}
{6: (210, 54)}
{6: (210, 58)}
{6: (210, 62)}
{6: (210, 66)}
{6: (210, 70)}
{6: (210, 74)}
{6: (210, 78)}
{6: (211, 80)}
{6: (211, 83), 7: (162, 18)}
{6: (211, 83), 7: (162, 18)}
{6: (212, 85), 7: (162, 18)}
{6: (212, 85), 7: (162, 16)}
{6: (212, 87), 7: (162, 16)}
{6: (212, 87), 7: (160, 25)}
{6: (2

{19: (19, 234), 30: (279, 196), 9: (289, 159), 27: (176, 62), 13: (38, 91), 28: (18, 130)}
{19: (19, 234), 30: (279, 201), 9: (289, 159), 27: (176, 62), 13: (38, 91), 28: (18, 130)}
{19: (19, 234), 30: (279, 201), 9: (289, 159), 27: (176, 62), 13: (38, 91), 28: (18, 130)}
{19: (19, 234), 30: (279, 201), 9: (289, 159), 27: (178, 67), 13: (38, 91), 28: (18, 130)}
{19: (19, 234), 30: (279, 201), 9: (289, 159), 27: (178, 67), 13: (38, 92), 28: (18, 130)}
{19: (19, 234), 30: (282, 205), 9: (289, 159), 27: (178, 67), 13: (38, 92), 31: (207, 247)}
{19: (19, 234), 30: (282, 205), 9: (291, 174), 27: (178, 67), 13: (38, 92), 31: (207, 247)}
{19: (19, 234), 30: (282, 205), 9: (291, 174), 27: (176, 72), 13: (38, 92), 31: (207, 247), 32: (31, 199)}
{19: (19, 234), 30: (282, 205), 9: (291, 174), 27: (176, 72), 13: (38, 92), 31: (207, 247), 32: (31, 199), 33: (204, 38)}
{31: (207, 247), 30: (282, 205), 9: (291, 174), 32: (31, 199), 27: (176, 72), 33: (204, 38), 13: (38, 92), 34: (11, 149), 35: (23, 1

{30: (288, 264), 9: (291, 173), 32: (29, 206), 27: (175, 142), 36: (206, 80), 13: (40, 83), 38: (16, 136)}
{30: (288, 264), 9: (291, 174), 32: (29, 206), 27: (175, 142), 36: (206, 80), 13: (40, 83), 38: (16, 136)}
{30: (288, 264), 9: (291, 174), 32: (29, 206), 27: (174, 148), 36: (206, 80), 13: (40, 83), 38: (16, 136)}
{30: (288, 264), 9: (291, 174), 32: (29, 206), 27: (174, 148), 36: (206, 80), 13: (40, 83), 38: (16, 136)}
{30: (288, 264), 9: (291, 174), 32: (29, 206), 27: (174, 148), 36: (206, 82), 13: (40, 83), 38: (16, 136)}
{30: (288, 264), 9: (291, 174), 32: (29, 206), 27: (174, 148), 36: (206, 82), 13: (39, 92), 38: (16, 136)}
{30: (288, 264), 9: (291, 174), 32: (29, 206), 27: (174, 148), 36: (206, 82), 13: (39, 92), 38: (16, 136)}
{30: (288, 264), 9: (291, 174), 32: (29, 206), 27: (174, 148), 36: (206, 82), 13: (40, 83), 38: (16, 136)}
{30: (289, 269), 9: (291, 174), 27: (174, 148), 32: (29, 206), 36: (206, 82), 13: (40, 83), 38: (16, 136)}
{30: (289, 269), 9: (291, 174), 27: (

{27: (164, 319), 50: (217, 227), 42: (292, 169), 45: (30, 208), 51: (224, 35), 13: (40, 82), 46: (18, 135)}
{27: (164, 319), 50: (215, 227), 42: (292, 169), 45: (30, 208), 51: (224, 35), 13: (40, 82), 46: (18, 135)}
{27: (164, 319), 50: (215, 227), 42: (292, 169), 45: (30, 208), 51: (224, 35), 13: (40, 82), 46: (18, 135)}
{27: (164, 319), 50: (215, 227), 42: (292, 169), 45: (30, 208), 51: (224, 35), 13: (40, 82), 46: (18, 135)}
{27: (164, 319), 50: (215, 227), 42: (292, 169), 45: (30, 208), 51: (224, 35), 13: (40, 93), 46: (18, 135)}
{27: (164, 319), 50: (215, 227), 42: (292, 169), 45: (30, 208), 51: (224, 35), 13: (40, 93), 46: (11, 150)}
{27: (164, 319), 50: (215, 227), 42: (292, 169), 45: (30, 208), 51: (224, 35), 13: (40, 82), 46: (11, 150)}
{27: (164, 332), 50: (215, 227), 42: (292, 169), 45: (30, 208), 13: (40, 82), 46: (11, 150)}
{27: (164, 332), 50: (214, 235), 42: (292, 169), 45: (30, 208), 13: (40, 82), 46: (11, 150)}
{27: (164, 332), 50: (214, 235), 42: (292, 169), 45: (30, 

{57: (116, 244), 58: (212, 245), 59: (166, 183), 61: (292, 169), 45: (32, 205), 52: (231, 109), 13: (39, 87), 62: (21, 23)}
{57: (116, 244), 58: (212, 245), 59: (166, 183), 61: (292, 169), 45: (32, 205), 52: (231, 109), 13: (39, 87), 62: (21, 23)}
{57: (116, 244), 58: (212, 245), 59: (166, 183), 61: (292, 169), 45: (32, 205), 52: (231, 116), 13: (39, 87), 62: (21, 23)}
{57: (116, 244), 58: (212, 245), 59: (166, 183), 61: (292, 169), 45: (32, 205), 52: (231, 116), 13: (41, 99), 62: (21, 23)}
{57: (116, 244), 58: (212, 245), 59: (166, 183), 61: (292, 169), 45: (32, 205), 52: (231, 116), 13: (38, 89), 62: (21, 23), 63: (25, 120)}
{57: (116, 243), 58: (212, 245), 59: (166, 183), 61: (292, 169), 45: (32, 205), 52: (231, 116), 13: (38, 89), 63: (25, 120)}
{57: (116, 243), 58: (213, 244), 59: (166, 183), 61: (292, 169), 45: (32, 205), 52: (231, 116), 13: (38, 89), 63: (25, 120)}
{57: (116, 243), 58: (213, 244), 59: (150, 182), 61: (292, 169), 45: (32, 205), 52: (231, 116), 13: (38, 89), 63: (

...

{342: (166, 245), 409: (271, 284), 407: (98, 54)}
{342: (166, 245), 409: (271, 284), 407: (98, 54)}
{342: (166, 245), 409: (271, 284), 407: (99, 57)}
{342: (166, 245), 409: (271, 284), 407: (99, 57)}
{342: (166, 245), 409: (271, 284), 407: (98, 61)}
{409: (271, 284), 407: (98, 61)}
{409: (271, 284), 407: (96, 69)}
{409: (271, 284), 407: (96, 69)}
{409: (271, 284), 407: (95, 74)}
{409: (272, 284), 407: (95, 74)}
{409: (272, 284), 407: (94, 77)}
{409: (272, 284), 407: (94, 77)}
{409: (272, 284), 407: (94, 83)}
{409: (272, 284), 407: (94, 83)}
{409: (272, 284), 407: (92, 89)}
{409: (272, 284), 407: (92, 89)}
{409: (272, 284), 407: (93, 97)}
{409: (272, 284), 407: (93, 97)}
{409: (272, 284), 407: (93, 100)}
{409: (272, 284), 407: (93, 100)}
{409: (272, 284), 407: (92, 106)}
{409: (272, 284), 407: (92, 106)}
{409: (272, 284), 407: (92, 121)}
{409: (272, 284), 407: (92, 121)}
{409: (272, 284), 407: (90, 121)}
{409: (272, 284), 407: (90, 121)}
{409: (272, 284), 407: (89, 127)}
{409: (272, 284

{420: (64, 267), 421: (275, 223), 409: (273, 284), 412: (275, 87), 411: (188, 43), 422: (17, 32), 423: (63, 294)}
{420: (64, 267), 421: (275, 223), 409: (273, 284), 412: (279, 100), 411: (188, 43), 422: (17, 32), 423: (63, 294)}
{420: (64, 267), 421: (275, 223), 409: (273, 284), 412: (279, 100), 411: (188, 46), 422: (17, 32), 423: (63, 294)}
{423: (61, 309), 409: (273, 284), 412: (279, 100), 411: (188, 46)}
{423: (61, 309), 409: (273, 284), 412: (279, 100), 411: (188, 46)}
{423: (61, 309), 409: (273, 284), 412: (279, 100), 411: (189, 50), 424: (295, 208)}
{423: (57, 324), 409: (273, 284), 424: (295, 208), 411: (189, 50)}
{423: (57, 324), 409: (273, 284), 424: (295, 208), 411: (189, 50)}
{423: (57, 324), 409: (273, 284), 424: (295, 208), 411: (189, 50)}
{423: (57, 324), 409: (273, 284), 424: (295, 208), 411: (189, 53), 425: (280, 107)}
{423: (56, 339), 409: (273, 284), 424: (295, 208), 425: (280, 107), 411: (189, 53), 426: (136, 11), 427: (17, 32)}
{423: (56, 339), 409: (273, 284), 424:

{432: (275, 284), 431: (210, 34), 433: (20, 28)}
{432: (275, 284), 431: (210, 37), 433: (20, 28)}
{432: (275, 284), 431: (210, 37), 433: (20, 28)}
{432: (275, 284), 431: (210, 37), 433: (20, 28)}
{432: (275, 284), 431: (211, 41), 433: (20, 28)}
{432: (275, 284), 431: (211, 41), 433: (20, 29), 434: (138, 10)}
{432: (275, 284), 431: (211, 41), 434: (138, 10), 433: (20, 29)}
{432: (275, 284), 431: (212, 45), 434: (138, 10), 433: (20, 29)}
{432: (275, 284), 431: (212, 45), 434: (138, 12), 433: (20, 29)}
{432: (275, 284), 431: (212, 45), 434: (138, 12), 433: (20, 29)}
{432: (275, 284), 431: (212, 45), 434: (138, 12), 433: (20, 29)}
{432: (275, 284), 431: (212, 48), 434: (138, 12), 433: (20, 29)}
{432: (275, 284), 431: (212, 48), 434: (138, 14), 433: (20, 29)}
{432: (275, 284), 431: (212, 48), 434: (138, 14), 433: (20, 29)}
{432: (275, 284), 431: (212, 48), 434: (138, 14), 433: (20, 29)}
{432: (275, 284), 431: (212, 52), 434: (138, 14), 433: (20, 29)}
{432: (275, 284), 431: (212, 52), 434: (

{432: (275, 283), 431: (232, 201), 434: (132, 160), 436: (93, 80), 433: (14, 49)}
{432: (275, 283), 431: (233, 208), 434: (132, 160), 436: (93, 80), 433: (14, 49)}
{432: (275, 283), 431: (233, 208), 434: (131, 167), 436: (93, 80), 433: (14, 49)}
{432: (275, 283), 431: (233, 208), 434: (131, 167), 436: (92, 85), 433: (14, 49)}
{432: (275, 283), 431: (233, 208), 434: (131, 167), 436: (92, 85), 433: (14, 49)}
{432: (275, 283), 431: (233, 208), 434: (131, 167), 436: (92, 85), 433: (14, 49)}
{432: (275, 283), 431: (234, 216), 434: (131, 167), 436: (92, 85), 433: (14, 49)}
{432: (275, 283), 431: (234, 216), 434: (131, 174), 436: (92, 85), 433: (14, 49)}
{432: (275, 283), 431: (234, 216), 434: (131, 174), 436: (91, 92), 433: (14, 49)}
{432: (275, 283), 431: (234, 216), 434: (131, 174), 436: (91, 92), 433: (14, 49)}
{432: (275, 283), 431: (235, 223), 434: (131, 174), 436: (91, 92), 433: (14, 49)}
{432: (275, 283), 431: (235, 223), 434: (131, 174), 436: (91, 92), 433: (14, 49)}
{432: (275, 283)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import cv2
cap = cv2.VideoCapture('highway.mp4')
algo = cv2.bgsegm.createBackgroundSubtractorMOG()

In [1]:
import cv2
import numpy as np
from time import sleep

min_width_rect = 80
min_height_rect = 80

offset = 6 # allow error between pixel
delay = 60 # FPS do video
carros = 0

count_line_pos = 550 # count line is position
detct = []

def central_handle(x,y,w, h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x + x1
    cy = y + y1
    return cx, cy


cap = cv2.VideoCapture("video.mp4")
algo = cv2.bgsegm.createBackgroundSubtractorMOG()

while True:
    ret, frame1 = cap.read()
    gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 5)
    
    # applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    counterShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame1, (25, count_line_pos), (1200,count_line_pos),(0,0,255), 3)
    
    for (i, c) in enumerate(counterShape):
        (x,y,w,h) = cv2.boundingRect(c)
        validar_contorno = (w >=min_width_rect) and (h >=min_height_rect)
        
        if not validar_contorno:
            continue
        
        cv2.rectangle(frame1, (x,y),(x+w, y+h),(0,255,0), 2)
        cv2.putText(frame1, "Vehicle" + str(carros), (x,y-20), cv2.FONT_HERSHEY_TRIPLEX, 1, (255,244,0),2)
        
        center = central_handle(x,y,w,h)
        detct.append(center)
        cv2.circle(frame1, center, 4, (0,0,255), -1)
        
        # Loop
        for (x,y) in detct:
            if y<(count_line_pos + offset) and y>(count_line_pos - offset):
                carros +=1
                cv2.line(frame1, (25, count_line_pos), (1200, count_line_pos), (0,255,0),3)
                detct.remove((x,y))
                print("Car is detected :" + str(carros))
        cv2.putText(frame1, "Vehicle Count :" + str(carros), (450,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,255),5)
    
    
    cv2.imshow("Final Video Analysis", frame1)
    if cv2.waitKey(1)==13:
        break

cv2.destroyAllWindows()
cap.release()   

Car is detected :1
Car is detected :2
Car is detected :3
Car is detected :4
Car is detected :5
Car is detected :6
Car is detected :7
Car is detected :8
Car is detected :9
Car is detected :10
Car is detected :11
Car is detected :12
Car is detected :13
Car is detected :14
Car is detected :15
Car is detected :16
Car is detected :17
Car is detected :18
Car is detected :19
Car is detected :20
Car is detected :21
Car is detected :22
Car is detected :23
Car is detected :24
Car is detected :25
Car is detected :26
Car is detected :27
Car is detected :28
Car is detected :29
Car is detected :30
Car is detected :31
Car is detected :32
Car is detected :33
Car is detected :34
Car is detected :35
Car is detected :36
Car is detected :37
Car is detected :38


KeyboardInterrupt: 